In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import json
import re


import unidecode #for removing accents from strings.


import os

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import remove_stopwords

from nltk.stem.porter import PorterStemmer


# Overview of the Data


The data we use for training our model are from two sources. 

The first is Arxiv.org's datadump on Kaggle. Arxiv is a online scientific preprint repository, and the dump contains the metadata for 1.7 million preprints uploaded to the site, including their titles, category, authors, and date uploaded. I labeled each category as either pure or applied and use this as one dataset to train my model on. 

Additionally, I scraped around 400,000 titles and their categories from 2001 to 2006 from Mathscinet. Mathscinet classifies papers using 61 supercategories, each with several subcategories for a total of 5000+ different categorical labels. Again, I labeled each supercategory as either being pure or applied. Below I discuss the labeling in more detail. 

We will use the Arxiv dataset for training Word2Vec embeddings, the reason being the dataset also contains abstracts (which supply much more text) and also vocabulary from areas outside of math, which will be important for classifying papers as pure/applied (indeed, earlier experiments just using math abstracts from arxiv to train Word2Vec yeilded worse performance). The dimension of the embeddings is 256 (I tried 512 before, but using PCA one could see the explained variance plateau a bit after 200).

In another notebook, we will train neural networks with the same architectures separately on the labeled arxiv titles and the labeled mathscinet titles. We keep them separate since, as the categorization rules are different between the two sites, we can't guarantee that our labeling of pure/applied will be consistent. 

Below we show how we cleaned and prepared the data and how we learned the Word2Vec embeddings. 

A word of warning, this takes a lot of time. If you wanted to run this or something similar for yourself, let the whole notebook run overnight.

We start first with the Arxiv dataset.


# Labelling Arxiv titles


The main categories for publications used by Arxiv are:

Computer Science
Physics
Mathematics
Economics
Quantitative Finance
Quantitative Biology
Statistics
Electrical Engineering and Systems Science

Within mathematics, there are several subcategories:



In [2]:
pure = ["math." + cat for cat in ["AC","AG","AP","AT","CA","CO","CT","CV","DG","DS","FA","GN","GR","GT","HO","KT",
                                  "LO","MG","MP","NT","OA","QA","RA","RT","SG","SP"]] + ["math-ph"]


applied = ["math." + cat for cat in ["IT","NA","OC","OR","ST","PR"]]

arxiv_math_categories = pure + applied


names = [
"Commutative Algebra",
"Algebraic Geometry",
"Analysis of PDEs",
"Algebraic Topology",
"Classical Analysis and ODEs",
"Combinatorics",
"Category Theory",
"Complex Variables",
"Differential Geometry",
"Dynamical Systems",
"Functional Analysis",
"General Topology",
"Group Theory",
"Geometric Topology",
"History and Overview",
"K-Theory and Homology",
"Logic",
"Metric Geometry",
"Mathematical Physics",
"Number Theory",
"Operator Algebras",
"Quantum Algebra",
"Rings and Algebras",
"Representation Theory",
"Symplectic Geometry",
"Spectral Theory",
    "Also Math Physics",
"Information Technology",
    "Numerical Analysis",
    "Optimization and Control",
    "Operations Research",
"Probability",
    "Statistics Theory"]

category_df = pd.DataFrame({"Category": names, "Abbreviation": arxiv_math_categories})
category_df

,Category,Abbreviation
0,Commutative Algebra,math.AC
1,Algebraic Geometry,math.AG
2,Analysis of PDEs,math.AP
3,Algebraic Topology,math.AT
4,Classical Analysis and ODEs,math.CA
5,Combinatorics,math.CO
6,Category Theory,math.CT
7,Complex Variables,math.CV
8,Differential Geometry,math.DG
9,Dynamical Systems,math.DS


There is also a General Math (math.GM) category, but since those topics could be of any category, we omit them.

Our focus will be on mathematics papers since that's what I'm most familiar with. We will consider two different classifiers: one is a multilabel classifier that tries to identify which of the above math categories belongs to, and we will also try to classify papers as pure or applied. For a  math paper to be "applied", I mean it is either in Statistics, Probability, Optimization and Control, Numerical Analysis, and Information Technology. 

My reasoning for this labelling is as follows: statitics and computer programming are highly saught-after skills, so Statistics, Numerical Analysis, and IT should be included. Depending on the kind of probability you do, it might be more "pure" or "applied" in the traditional sense, but I put it in the "applied" category since finance jobs seek people who understand concepts like Brownian Motion and Stochastic PDEs. By searching on a job site Monster.com one sees there are jobs asking for people who know control theory or optimization, so I consider this category to be applied as well. While mathematical physics would seem to be an applied subject, it is more on the theoretical side.

There are a few weaknesses in this labeling scheme which I list below that perhaps a future project could improve upon.

Now just because a paper is in any other field does not indicate that the authors don't know or use programming. I've known people to use programming to study pure mathematics problems in algebra and PDEs. However, someone graduating with a PhD in this field is not guaranteed to have demonstrated in their thesis work that they can program. 

There are subjects where there is more crossover: some papers in combinatorics are crosslisted in computer science and study algorithms or discrete geometry. My decision to label this as "pure" comes from googling the question "I did a PhD in combinatorics and want to switch to industry" and seeing what discussions arise. It seems that most combinatorics is still quite abstract, and there are students asking this question who are advised that, while having a math PhD is desirable and combinatorics sure helps in programming, they should supplement this with learning to program. 

Another weakness in the classification is that I classified any paper categorized as a non-math topic as "applied". This was initially with the assumption that working in physics implies some numerate or statistical ability, however, not all physics students specialize in experimental physics (there is also theoretical/mathematical physics), and I have seen articles online of physicists charting their transition from research to data science, which indicates that being, so apparently moving out of physics is not 100% straightforward. However, I still labeled physics as "applied" as I do not have the knowledge to sift through the subcategories and distinguish which are more employable than others.


# Cleaning the Arxiv metadata: all about the $

The Arxiv metadata comes as a json file. We convert it to some csv files for convenience with some additional features:

First, we add a separate column for whether the paper is "pure" or "applied" by choosing some math categories to designate as "pure" and label everything else as applied. 

Furthermore, we create a new column for the primary category (so for each item, we look  at the "categories" item and select the first listed label. 

At the same time, we will make separate csv files consisting only of preprocessed titles and abstracts (and separate ones for those that are math papers), along with their categories and the pure/applied label. 

The preprocessing involves some usual text cleaning methods but also some that are particular to scientific papers. 

Firstly, many titles and abstracts use LaTeX commands. Instead of trying to treat the math written in the latex commands as words, we will just filter them out of the text. Commands are enclosed in dollar signs, so this is easy. 

Another issue are accents, since some people write these either using an actual unicode character like ö, they may use the LaTeX equivalent, which is \"o (without dollar signs). We need to filter these out. Luckily, there is usually a slash followed by another character (some exceptions are \o{} for ø or {\i} for $\iota$, but we will allow those to save on computational cost) and we replace them with an empty character (after removing newlines \n with a space).

Apart from that is the usual stuff: remove stopwords, deaccenting, stemming, etc. 

Experiments earlier that just removed all symbols still performed well, but the extra processing improved performance by several percentage points. 



In [3]:

#The following processes a string into a list of tokens to pass into gensim phraser. 


stemmer = PorterStemmer()
def get_string_tokens(string):

    string = re.sub("(\$.+?\$)","",string, flags = re.DOTALL) #Removes Latex
    string = remove_stopwords(string.lower()) #Removes Stopwords and lowers all characters
    string = unidecode.unidecode(string) #Replaces accents, e.g. ö --> o
    string = string.replace("\n"," ") #Replace newlines with just a space
    string = re.sub("(\\\\+.)","",string) #This removes some remaining latex often used for accents, like Carath\'eodory --> Caratheodory.
    string = re.sub("([\{\}]+)","",string) #Some accents are written  Carath\'{e}odory, which becomes Carath{e}odory by the previous step, so we remove the brackets.
    tokens = simple_preprocess(string, max_len=100) #Removes symbols and returns list of tokens
    tokens = [stemmer.stem(token) for token in tokens] #stems tokens
    
    return tokens

    
    

def get_metadata(data_file):
    with open(data_file, 'r') as f:
        for line in f:
            yield json.loads(line)
                
def create_arxiv_csv(data_file = 'arxiv-metadata-oai-snapshot.json', pure=pure):


    """ Using `yield` to load the JSON file in a loop to prevent Python memory issues if JSON is loaded directly"""
    
    
    

    get = get_metadata(data_file)

    i=0
    header = True
    mode= "w"
    for article in get:

        i+=1
        if "math.GM" in article["categories"]:
            continue
        else:


            categories = article["categories"].split(" ")

            main_category = categories[0]
            if main_category in pure:
#                 if all([category in pure for category in categories]):
                is_pure = 1
            else:
                is_pure = 0

            
            title = article["title"]
            
            processed_title = " ".join(get_string_tokens(title))
            
            abstract = article["abstract"]
            
            processed_abstract = " ".join(get_string_tokens(abstract))
            


            article = pd.DataFrame({"title":[article["title"]],
                                    "abstract":[article["abstract"]],
                                    "processed_abstract":[processed_abstract],
                                    "processed_title":[processed_title],
                                    "date":[article["versions"][0]["created"]],
                                    "categories": [article["categories"]],
                                    "main_category":[main_category],
                                    "pure": [is_pure]
                                       })

            article[["title","abstract","date","categories","main_category","pure"]].to_csv("arxiv.csv",
                                                                                               header=header,
                                                                                               mode=mode,
                                                                                              index=False)
            
            article[["processed_title","main_category","pure"]].to_csv("processed_titles.csv",
                                                                          header=header,
                                                                          mode=mode,  index=False)
            
            article[["processed_abstract","main_category","pure"]].to_csv("processed_abstracts.csv",
                                                                          header=header,
                                                                          mode=mode, index=False)
            
            math = article[article["main_category"].str.contains("math.", regex=False)]
            
            math[["processed_title","categories","main_category","pure"]].to_csv("processed_math_titles.csv",
                                                                          header=header,
                                                                          mode=mode,  index=False)
            
            math[["processed_abstract","main_category","pure"]].to_csv("processed_math_abstracts.csv",
                                                                          header=header,
                                                                          mode=mode, index=False)
            

            print(f"\r{i} articles processed...",end="")
            
            header=False
            mode = "a"
            

    print("File written successfully")



create_arxiv_csv(data_file = 'arxiv-metadata-oai-snapshot.json', pure=pure)



1789907 articles processed...File written successfully


# Bigrams



After that we will learn bigrams from the titles/abstracts. Since these are stored in large files, we need to pass a generator to Gensim's Phrases model to learn them instead of storing them in memory. To save on time, we will create a generator that also does the preprocessing into csv files as described above. 



In [8]:


#This class iterates through the arxiv file, yielding processed titles/abstracts

#We will also make another file with the processed abstracts so we don't have to do the processing again

class get_text_gen:
    def __init__(self, file_name = "processed_abstracts.csv", column = "processed_abstract", preprocessed=True):
        self.file_name = file_name
        self.column = column
        
        if preprocessed == True:
            self.process = lambda x: x.split(" ")
        else:
            self.process = get_string_tokens
        

    def __iter__(self):
        
        i=0
        
        file = pd.read_csv(self.file_name,chunksize=10000)
        
        for chunk in file:
        
            texts = chunk[self.column]

            for text in texts:
                yield self.process(text)
                i+=1
                print(f"\r{i} articles yielded...",end="")
            
            
                

generator = get_text_gen()


#If passed a file_name and generator, this will learn the bigrams from the strings yielded from the 
#generator and then save the model as file_name; if no generator is given, it will instead load
#a previously saved bigram model from file_name
def get_bigram(file_name, generator=None):

    if generator == None:
        bigram = gensim.models.phrases.Phrases.load(file_name)
        print("Load successful!")
        return bigram
        
    else:
        bigram = gensim.models.phrases.Phrases(generator, min_count=10, threshold=2)

        print("Finished finding Bigrams!")
        
        bigram.save(file_name)
        
        print("Model Saved!")
        return bigram

bigrammer = get_bigram(file_name = "bigram.model",generator=generator)


#For a sanity check, let's see how the bigrammer combined with the processor handles some common strings

string = "Markov Chain Monte Carlo, Fourier Transforms, noncommutative algebra, rectifiable sets"
tokens = get_string_tokens(string)
bigrammer[tokens]

382 articles yielded...

803 articles yielded...

1251 articles yielded...

1768 articles yielded...

2330 articles yielded...

...

8803 articles yielded...

9310 articles yielded...

9857 articles yielded...

1786970 articles yielded...Finished finding Bigrams!
Model Saved!


['markov_chain',
 'mont_carlo',
 'fourier_transform',
 'noncommut_algebra',
 'rectifi',
 'set']

# Word2Vec Embedding

In this section we learn the Word2Vec Embeddings. Again, we use the model from Gensim. First, we make a generator that will feed the processed abstracts to the model.

We only use the abstracts from Arxiv to learn the vectors. We don't use the mathscinet title since a) there is less text and b) the Arxiv abstracts contain terminology from many different fields other than mathematics, which is important if we want to identify whether a title is applied. 

The dimension of the word2vec embeddings will be set to 256. We tried with higher values like 512, but PCA showed that the explained variance began to plateau aound 200, so it didn't seem like we would gain much from a larger ambient space.



In [9]:
class get_bigrammed_texts:
    def __init__(self, file_name = "processed_abstracts.csv", 
                 column = "processed_abstract", 
                 bigrammer=bigrammer,
                preprocessed = True):
        
        self.file_name = file_name
        self.column = column
        self.epoch=0
        
        if preprocessed == True:
            self.process = lambda x: bigrammer[x.split(" ")]
        else:
            self.preprocess = lambda x : bigrammer[get_sring_tokens(x)]

    
    def __iter__(self):
        
        self.epoch+=1
        
        i=0
        
        df = pd.read_csv(self.file_name,chunksize=10000)
        
        for chunk in df:
            for token in chunk[self.column]:
                yield self.process(token)
                print(f"\r{i} articles of epoch {self.epoch}...",end="")
                i+=1
                
                

bigram_generator = get_bigrammed_texts()



Now we feed this generator into the Word2Vec Model

In [10]:



#Again, if a bigram_generator is given, it will learn the word2vec representations and save the model
#as file_name; otherwise it will load a previously saved model from file_name
#We can also pass other kwargs to the Gensim's Word2Vec model, like the number of epochs, window size, etc.

def get_word2vec(file_name, bigram_generator=None, **kwargs):
    
    if bigram_generator==None:
        
        model = gensim.models.Word2Vec.load(file_name)
        print("File loaded!")
        
        return model
    
    else:
        print("Learning vectors...")
        
        model = gensim.models.word2vec.Word2Vec(sentences=bigram_generator, **kwargs)
        
        print("Learned vectors!")
        
        model.save(file_name)
        
        print("Model saved as " + file_name)
        
        return model
    
wv_model = get_word2vec(file_name = "word2vec.model",window=10, sg=1, size=256, bigram_generator = bigram_generator)


Learning vectors...
1786969 articles of epoch 6...Learned vectors!
Model saved as word2vec.model


Again, as a sanity check, let's look at the most similar vectors to some common terms. Note that there are still some misspellings of schrodinger that have been created because of accents getting turned into spaces, but the bigrammer still groups together the broken pieces into bigrams and word2vec still recognizes them as being similar words. 

In [13]:
def process_word(string):
    return " ".join(bigrammer[get_string_tokens(string)])

for word in ["schrodinger","monte carlo", "metric space","hausdorff dimension", ]:
    print(wv_model.most_similar(process_word(word)))

/Users/jazzam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """


[('schroeding', 0.8230994939804077), ('schroding_equat', 0.7191005945205688), ('schr_oding', 0.7140465974807739), ('schroeding_equat', 0.692336916923523), ('schrdinger', 0.6703927516937256), ('schoeding', 0.6639249324798584), ('schoding', 0.6581615209579468), ('stargenvalu', 0.6550089120864868), ('shroding', 0.6546859741210938), ('doebner_goldin', 0.6540631055831909)]
[('mc_simul', 0.733250081539154), ('simul', 0.7229532599449158), ('rathsman', 0.6707162857055664), ('montecarlo', 0.6619164943695068), ('carlo_mc', 0.6618112325668335), ('ekhara', 0.658603847026825), ('herwig_mont', 0.6425133943557739), ('gener_phokhara', 0.6417258977890015), ('qcdin', 0.6397531032562256), ('multimagnet', 0.6395081281661987)]
[('urysohn_univers', 0.7157913446426392), ('yoneda_complet', 0.6919934749603271), ('equip_wasserstein', 0.6906655430793762), ('pseudometr_space', 0.6848750114440918), ('urysohn_metric', 0.6839586496353149), ('ultraextens', 0.6832410097122192), ('isometr_emb', 0.6811697483062744), ('n

# The Mathscinet data

I won't make the scraped mathscinet data here, but the preprocessing of the titles is exactly the same. However, we shold mention how we did the labeling. 

Mathscinet classifies papers according to the MSC classification system. Below we list the main categories (minus the general mathematics category) and partition them into pure and applied.

In [1]:
applied = [
       
       'probability theory and stochastic processes', 'statistics',
       'numerical analysis', 'computer science',
       'mechanics of particles and systems',
       'mechanics of deformable solids', 'fluid mechanics',
       'optics, electromagnetic theory',
       'classical thermodynamics, heat transfer', 
       'statistical mechanics, structure of matter',
       'astronomy and astrophysics', 'geophysics',
       'operations research, mathematical programming',
       'game theory, economics, social and behavioral sciences',
       'biology and other natural sciences', 'systems theory; control',
       'information and communication, circuits', 'mathematics education',      
        'calculus of variations and optimal control; optimization']

pure = ['difference and functional equations','history and biography', 'dynamical systems and ergodic theory', 
        'mathematical logic and foundations',
       'combinatorics', 'order, lattices, ordered algebraic structures',
       'general algebraic systems', 'number theory',
       'field theory and polynomials', 'commutative rings and algebras',
       'algebraic geometry',
       'linear and multilinear algebra; matrix theory',
       'associative rings and algebras',
       'nonassociative rings and algebras',
       'category theory; homological algebra', '$k$-theory',
       'group theory and generalizations',
       'topological groups, lie groups', 'real functions',
       'measure and integration', 'functions of a complex variable',
       'potential theory',
       'several complex variables and analytic spaces',
       'special functions (33-xx deals with the properties of functions as functions)',
       'ordinary differential equations',
       'partial differential equations','relativity and gravitational theory', 'quantum theory', 'geometry', 'convex and discrete geometry',
       'differential geometry', 'general topology', 'algebraic topology',
       'manifolds and cell complexes',
       'global analysis, analysis on manifolds', 'sequences, series, summability', 'approximations and expansions',
       'fourier analysis', 'abstract harmonic analysis',
       'integral transforms, operational calculus', 'integral equations',
       'functional analysis', 'operator theory']





Again, there isn't a clear cut between pure and applied. For example, "abstract harmonic analysis" is unambiguously pure, but "calculus of variations and optimal control" is a mix (with some very pure results and very applied results in the same category), but we'll play it safe and classify that as applied. 

The last preprocessing we did with the mathscinet database is filter out non-English titues using Googles langdetect package. 